Imports take a pretty long time

In [1]:
import re
import pandas as pd
from datetime import datetime
from scrape import get_data
import spacy
from spacy import displacy
from bertopic import BERTopic

Loading entity extraction model

In [2]:
entt = spacy.load("en_core_web_sm")

Creating a topic model

In [3]:
topic_model = BERTopic(verbose=True , calculate_probabilities=True , nr_topics="auto")

Loading the data

In [4]:
texts_df = pd.read_csv('/Users/ilyafeldman/Desktop/Coding/Codes/Telegram/telegraph/scrapevis/4800 sputnik telegram messages.csv')

Cleaning the data

In [5]:
texts_df.text = texts_df.apply(lambda row: re.sub(r"http\S+", "", row.text).lower(), 1)
#texts_df.text = texts_df.apply(lambda row: " ".join(filter(lambda x:x[0]!="@", row.text.split())), 1)
texts_df.text = texts_df.apply(lambda row: " ".join(re.sub("[^a-zA-Z]+", " ", row.text).split()), 1)
timestamps = texts_df.date.to_list()
texts = texts_df.text.to_list()

Loading the data to the topic model

In [6]:
topics, probs = topic_model.fit_transform(texts)

Batches: 100%|██████████| 151/151 [01:59<00:00,  1.26it/s]
2021-09-23 12:33:45,332 - BERTopic - Transformed documents to Embeddings
2021-09-23 12:34:05,092 - BERTopic - Reduced dimensionality with UMAP
2021-09-23 12:34:08,682 - BERTopic - Clustered UMAP embeddings with HDBSCAN
2021-09-23 12:34:39,300 - BERTopic - Reduced number of topics from 143 to 66


In [7]:
descs = []
for topic in topics:
    desc = topic_model.get_topic(topic)
    descs.append(desc)

In [8]:
text_topic = []

text_topic.extend([list(a) for a in zip(texts, topics , descs)])

In [9]:
text_topic_df = pd.DataFrame(text_topic, columns=['text' , 'topic id' , 'topic description'])

In [11]:
for doc in text_topic_df['text']:
    doc = nlp(doc)
    for ent in doc.ents:
        print(ent.text , ' - ' , spacy.explain(ent.label_))
    displacy.render(doc, style="ent")

NameError: name 'nlp' is not defined